In [4]:
import numpy as np
import pandas as pd
import datetime
import requests
from bs4 import BeautifulSoup
import copy

In [12]:
df_val = pd.read_csv('../Data/Player Valuation/player_valuations.csv')
df_players = pd.read_csv('../Data/Player Valuation/players.csv')
df_clubs = pd.read_csv('../Data/Player Valuation/clubs.csv')


In [16]:
df_val.head()

,player_id,last_season,datetime,date,dateweek,market_value_in_eur,n,current_club_id,player_club_domestic_competition_id
0,3132,2013,2003-12-09 00:00:00,2003-12-09,2003-12-08,400000,1,126,TR1
1,6893,2012,2003-12-15 00:00:00,2003-12-15,2003-12-15,900000,1,984,GB1
2,10,2015,2004-10-04 00:00:00,2004-10-04,2004-10-04,7000000,1,398,IT1
3,26,2017,2004-10-04 00:00:00,2004-10-04,2004-10-04,1500000,1,16,L1
4,65,2015,2004-10-04 00:00:00,2004-10-04,2004-10-04,8000000,1,1091,GR1


In [15]:
df_players.head()

,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,foot,height_in_cm,market_value_in_eur,highest_market_value_in_eur,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name
0,598,Timo,Hildebrand,Timo Hildebrand,2014,24,timo-hildebrand,Germany,Worms,Germany,...,NaN,NaN,NaN,10000000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/timo-hildebran...,L1,Eintracht Frankfurt
1,670,Martin,Petrov,Martin Petrov,2012,714,martin-petrov,Bulgaria,Vratsa,Bulgaria,...,NaN,NaN,NaN,12000000.0,NaN,IFM,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/martin-petrov/...,ES1,RCD Espanyol Barcelona
2,1323,Martin,Amedick,Martin Amedick,2012,24,martin-amedick,Germany,Paderborn,Germany,...,NaN,NaN,NaN,2750000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/martin-amedick...,L1,Eintracht Frankfurt
3,3195,Jermaine,Pennant,Jermaine Pennant,2013,512,jermaine-pennant,England,Nottingham,England,...,right,173.0,NaN,10500000.0,NaN,Andrew Sky,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/jermaine-penna...,GB1,Stoke City
4,3259,Damien,Duff,Damien Duff,2013,931,damien-duff,Ireland,Ballyboden,Ireland,...,left,177.0,NaN,17000000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/damien-duff/pr...,GB1,Fulham FC


In [14]:
df_clubs.head()

,club_id,club_code,name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,url
0,105,sv-darmstadt-98,Sportverein Darmstadt 1898 e. V.,L1,NaN,30,26.4,11,36.7,1,Merck-Stadion am Böllenfalltor,17810,€-1.60m,NaN,2023,https://www.transfermarkt.co.uk/sv-darmstadt-9...
1,11127,ural-ekaterinburg,FK Ural Yekaterinburg,RU1,NaN,25,28.2,13,52.0,5,Yekaterinburg Arena,23000,€-770k,NaN,2023,https://www.transfermarkt.co.uk/ural-ekaterinb...
2,114,besiktas-istanbul,Beşiktaş Jimnastik Kulübü,TR1,NaN,32,27.4,16,50.0,13,Tüpraş Stadyumu,42590,€-14.50m,NaN,2023,https://www.transfermarkt.co.uk/besiktas-istan...
3,12,as-rom,Associazione Sportiva Roma,IT1,NaN,26,27.4,17,65.4,17,Olimpico di Roma,73261,+€65.20m,NaN,2023,https://www.transfermarkt.co.uk/as-rom/startse...
4,148,tottenham-hotspur,Tottenham Hotspur Football Club,GB1,NaN,29,26.2,21,72.4,22,Tottenham Hotspur Stadium,62062,€-130.40m,NaN,2023,https://www.transfermarkt.co.uk/tottenham-hots...


In [17]:
df_val_player = pd.merge(df_players, df_val, on='player_id')

In [36]:
df_val_player['date'] = pd.to_datetime(df_val_player['date'])
df_val_player.reset_index().set_index('name')
df_val_player.keys()

Index(['player_id', 'first_name', 'last_name', 'name', 'last_season_x',
       'current_club_id_x', 'player_code', 'country_of_birth', 'city_of_birth',
       'country_of_citizenship', 'date_of_birth', 'sub_position', 'position',
       'foot', 'height_in_cm', 'market_value_in_eur_x',
       'highest_market_value_in_eur', 'contract_expiration_date', 'agent_name',
       'image_url', 'url', 'current_club_domestic_competition_id',
       'current_club_name', 'last_season_y', 'datetime', 'date', 'dateweek',
       'market_value_in_eur_y', 'n', 'current_club_id_y',
       'player_club_domestic_competition_id'],
      dtype='object')

In [37]:
df_val_player.sort_values(by='date', ascending=False)

,player_id,first_name,last_name,name,last_season_x,current_club_id_x,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,current_club_domestic_competition_id,current_club_name,last_season_y,datetime,date,dateweek,market_value_in_eur_y,n,current_club_id_y,player_club_domestic_competition_id
74579,405676,Oliver,Bias,Oliver Bias,2018,23826,oliver-bias,Germany,Johanngeorgenstadt,Philippines,...,L1,RB Leipzig,2018,2023-12-19 00:00:00,2023-12-19,2023-12-18,50000,1,23826,L1
430420,718115,NaN,Zini,Zini,2023,2441,zini,Angola,Cazenga,Angola,...,GR1,AEK Athens,2023,2023-09-21 00:00:00,2023-09-21,2023-09-18,300000,1,2441,GR1
276409,96790,Sebastian,Mladen,Sebastian Mladen,2023,6418,sebastian-mladen,Romania,Calafat,Romania,...,GR1,Panetolikos GFS,2023,2023-09-21 00:00:00,2023-09-21,2023-09-18,500000,1,6418,GR1
126125,335725,NaN,Shapi Suleymanov,Shapi Suleymanov,2023,605,shapi-suleymanov,Russia,Makhachkala,Russia,...,GR1,Aris Thessaloniki,2023,2023-09-21 00:00:00,2023-09-21,2023-09-18,2000000,1,605,GR1
223138,88348,Alexandros,Paschalakis,Alexandros Paschalakis,2023,683,alexandros-paschalakis,Greece,Athens,Greece,...,GR1,Olympiacos Piraeus,2023,2023-09-21 00:00:00,2023-09-21,2023-09-18,800000,1,683,GR1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268490,12503,Souleymane,Diawara,Souleymane Diawara,2014,417,souleymane-diawara,Senegal,Gabou,Senegal,...,FR1,OGC Nice,2014,2004-10-04 00:00:00,2004-10-04,2004-10-04,2500000,1,417,FR1
29801,9694,Efthymios,Kouloucheris,Efthymios Kouloucheris,2013,169,efthymios-kouloucheris,Greece,Karditsa,Greece,...,GR1,Panionios Athens,2013,2004-10-04 00:00:00,2004-10-04,2004-10-04,300000,1,169,GR1
268513,12665,Keith,Lasley,Keith Lasley,2016,987,keith-lasley,Scotland,Glasgow,Scotland,...,SC1,Motherwell FC,2016,2004-10-04 00:00:00,2004-10-04,2004-10-04,50000,1,987,SC1
105111,6893,Gabriel,Tamas,Gabriel Tamas,2012,984,gabriel-tamas,Romania,Brașov,Romania,...,GB1,West Bromwich Albion,2012,2003-12-15 00:00:00,2003-12-15,2003-12-15,900000,1,984,GB1


In [56]:
new_date = datetime.datetime(2023, 1, 1)
df_val_player.loc[(df_val_player['name'] == 'Victor Osimhen') & (df_val_player['date'] > new_date)].sort_values(by='date', ascending=False)


,player_id,first_name,last_name,name,last_season_x,current_club_id_x,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,current_club_domestic_competition_id,current_club_name,last_season_y,datetime,date,dateweek,market_value_in_eur_y,n,current_club_id_y,player_club_domestic_competition_id
435575,401923,Victor,Osimhen,Victor Osimhen,2023,6195,victor-osimhen,Nigeria,Lagos,Nigeria,...,IT1,SSC Napoli,2023,2023-06-15 00:00:00,2023-06-15,2023-06-12,120000000,1,6195,IT1
435574,401923,Victor,Osimhen,Victor Osimhen,2023,6195,victor-osimhen,Nigeria,Lagos,Nigeria,...,IT1,SSC Napoli,2023,2023-03-21 00:00:00,2023-03-21,2023-03-20,100000000,1,6195,IT1


In [24]:
def nan_values(df: pd.DataFrame):
    cols = df.columns.values.tolist()
    rows = df.index.values.tolist()
    for col in cols:
        if df[col].isna().sum() == 0:
            pass
        else:
            print(f"for {col}: {df[col].isna().sum()} NaN values")


In [60]:
checkdate = datetime.datetime(2023, 6, 1)
df_val_player_new = df_val_player[['player_id', 'name', 'market_value_in_eur_y', 'date']]
df_val_player_new = df_val_player_new.loc[(df_val_player['date'] > checkdate)]

In [67]:
df_val_player_new.loc[df_val_player_new['name'] == 'Kylian Mbappé']

,player_id,name,market_value_in_eur_y,date
73019,342229,Kylian Mbappé,180000000,2023-06-27


In [63]:
nan_values(df_val_player_new)

In [3]:
url_stats = 'https://fbref.com/en/comps/Big5/2019-2020/stats/players/2022-2023-Big-5-European-Leagues-Stats'
url_mics  = 'https://fbref.com/en/comps/Big5/2019-2020/misc/players/2022-2023-Big-5-European-Leagues-Stats' 
url_shoot = 'https://fbref.com/en/comps/Big5/2019-2020/shooting/players/2022-2023-Big-5-European-Leagues-Stats'
url_pass  = 'https://fbref.com/en/comps/Big5/2019-2020/passing/players/2022-2023-Big-5-European-Leagues-Stats'
url_def = 'https://fbref.com/en/comps/Big5/2019-2020/defense/players/2022-2023-Big-5-European-Leagues-Stats'
url_crea = 'https://fbref.com/en/comps/Big5/2019-2020/gca/players/2022-2023-Big-5-European-Leagues-Stats'

response = requests.get(url_mics)
soup = BeautifulSoup(response.text,"html.parser")
table = soup.find('table') 
headers = [ 'Player','Nation','Pos','Squad','Comp','Age','Born','90s','CrdY','CrdR','2CrdY','Fls','Fld','Off','Crs','Int','TklW','PKwon','PKcon','OG','Recov','Won','Lost','Won%','Matches']
df_misc = pd.DataFrame(columns=headers)
i=0
for j in table.find_all('tr')[1:]:
    row_data = j.find_all('td')
    row = [tr.text for tr in row_data]
    if row != []:
        df_misc.loc[i] = row
        i=i+1
df_misc

,Player,Nation,Pos,Squad,Comp,Age,Born,90s,CrdY,CrdR,...,Int,TklW,PKwon,PKcon,OG,Recov,Won,Lost,Won%,Matches
0,Patrick van Aanholt,nl NED,DF,Crystal Palace,eng Premier League,28,1990,27.9,0,0,...,45,23,0,0,1,176,16,16,50.0,Matches
1,Max Aarons,eng ENG,DF,Norwich City,eng Premier League,19,2000,36.0,7,0,...,37,30,1,0,0,178,22,49,31.0,Matches
2,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,31,1987,28.0,0,0,...,47,33,0,1,0,140,85,51,62.5,Matches
3,Suleiman Abdullahi,ng NGA,"FW,MF",Union Berlin,de Bundesliga,22,1996,1.7,1,0,...,1,0,0,0,0,7,6,11,35.3,Matches
4,Mehdi Abeid,dz ALG,MF,Nantes,fr Ligue 1,26,1992,20.2,5,0,...,23,29,0,1,0,153,13,17,43.3,Matches
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2727,Szymon Żurkowski,pl POL,MF,Fiorentina,it Serie A,21,1997,0.1,0,0,...,0,0,0,0,0,1,1,0,100.0,Matches
2728,David Zurutuza,es ESP,MF,Real Sociedad,es La Liga,33,1986,1.7,0,0,...,4,3,0,0,0,9,4,10,28.6,Matches
2729,Martin Ødegaard,no NOR,MF,Real Sociedad,es La Liga,20,1998,28.1,3,0,...,17,19,0,0,0,157,10,25,28.6,Matches
2730,Filip Đuričić,rs SRB,"MF,FW",Sassuolo,it Serie A,27,1992,18.1,6,0,...,8,14,1,1,0,75,11,20,35.5,Matches


In [14]:
df_stats_players = copy.deepcopy(df_misc)
df_stats_players.keys()

Index(['Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Born', '90s',
       'CrdY', 'CrdR', '2CrdY', 'Fls', 'Fld', 'Off', 'Crs', 'Int', 'TklW',
       'PKwon', 'PKcon', 'OG', 'Recov', 'Won', 'Lost', 'Won%', 'Matches'],
      dtype='object')

In [12]:
df_stats_players.loc[df_stats_players['Player'] == 'Aaron Ramsdale']

,Player,Nation,Pos,Squad,Comp,Age,Born,90s,CrdY,CrdR,...,Int,TklW,PKwon,PKcon,OG,Recov,Won,Lost,Won%,Matches
2060,Aaron Ramsdale,eng ENG,GK,Bournemouth,eng Premier League,21,1998,37.0,0,0,...,0,3,0,0,0,17,10,4,71.4,Matches
